#### Script to Predict the output on new observations

In [13]:
# !pip install matplotlib==3.9.2
# !pip install numpy==2.1.2
# !pip install pandas==2.2.3
# !pip install scikit_learn==1.5.2
# !pip install seaborn==0.13.2

In [26]:
import pandas as pd
import numpy as np
import pickle
from zipfile import ZipFile

pd.options.mode.copy_on_write = True 

In [15]:
zipfile = ZipFile('data/dataset.zip', 'r')
zipfile.namelist()

['test.csv',
 '__MACOSX/',
 '__MACOSX/._test.csv',
 'train.csv',
 '__MACOSX/._train.csv',
 'test_data_subset.csv',
 '__MACOSX/._test_data_subset.csv']

In [16]:
test_data = pd.read_csv(zipfile.open("test_data_subset.csv"))    

In [18]:
#load the columns to drop file
columns_to_drop=pd.read_csv("data/columns_to_drop.csv")
columns_to_Retain = list(set(test_data.columns.values) - set(columns_to_drop.colnames.values))
test_data = test_data[columns_to_Retain]

column_datatypes = test_data.dtypes
categorical_columns = list(column_datatypes[column_datatypes=="object"].index.values)

In [19]:
#Transfor the categorical columns by loading the fit encodings on training data

for cf1 in categorical_columns:
    filename = 'data/sav_files/' + cf1+".sav"
    le = pickle.load(open(filename, 'rb'))
    
    #if an new classes is observed, set it to the 0 class
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    test_data[cf1]=test_data[cf1].apply(lambda x: le_dict.get(x, -1))

In [20]:
test_data_id = test_data['id']
test_data = test_data.drop('id',axis=1)

In [21]:
Column_datatypes= test_data.dtypes
Integer_columns = list(Column_datatypes.where(lambda x: x =="int64").dropna().index.values)
#convert the int64 columns categorical
test_data[Integer_columns] = test_data[Integer_columns].astype('category',copy=False)

In [23]:
#load the saved model and predict on the test data
tunedmodel_rf = pickle.load(open("models/tunedmodel_rf", 'rb'))

In [27]:
expected_features = tunedmodel_rf.feature_names_in_
array = np.array(expected_features)
print("Expected features:", array)

Expected features: ['cat1' 'cat15' 'cat22' 'cat32' 'cat73' 'cat75' 'cat77' 'cat88' 'cat89'
 'cat90' 'cat93' 'cat94' 'cat95' 'cat96' 'cat99' 'cat100' 'cat101'
 'cat102' 'cat103' 'cat104' 'cat105' 'cat108' 'cat109' 'cat110' 'cat112'
 'cat113' 'cat115' 'cat116' 'cont1' 'cont2' 'cont3' 'cont4' 'cont5'
 'cont7' 'cont8' 'cont11' 'cont13' 'cont14']


In [28]:
test_data = test_data.reindex(array, axis=1)
test_data.columns

Index(['cat1', 'cat15', 'cat22', 'cat32', 'cat73', 'cat75', 'cat77', 'cat88',
       'cat89', 'cat90', 'cat93', 'cat94', 'cat95', 'cat96', 'cat99', 'cat100',
       'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat108', 'cat109',
       'cat110', 'cat112', 'cat113', 'cat115', 'cat116', 'cont1', 'cont2',
       'cont3', 'cont4', 'cont5', 'cont7', 'cont8', 'cont11', 'cont13',
       'cont14'],
      dtype='object')

In [29]:
Y_test_predict = tunedmodel_rf.predict(test_data)
test_data['predict_loss']=Y_test_predict
test_data['id']=test_data_id
test_data = test_data[['id','predict_loss']]

In [30]:
test_data

,id,predict_loss
0,4,7.790048
1,6,7.682504
2,9,9.012571
3,12,8.175791
4,15,7.047276
